In [30]:
import z3
from lib.ts import Ts
from lib.prove import prove_inductive

In [31]:
# The smart contract as a transition system.
ts = Ts('melon')

BASE_UNITS = 10 ** 18
INFLATION_ENABLE_DATE = 1551398400
INITIAL_TOTAL_SUPPLY = 932613 * BASE_UNITS

sender = z3.BitVec('sender', 16)
account_burn = z3.BitVec('account_burn',16)
value_burn = z3.BitVec('value_burn',256)
account_burnFrom = z3.BitVec('account_burnFrom',16)
value_burnFrom = z3.BitVec('value_burnFrom',256)
from_transferFrom = z3.BitVec('from_transferFrom',16)
to_transferFrom = z3.BitVec('to_transferFrom',16)
value_transferFrom = z3.BitVec('value_transferFrom',256)
initialReceiver = z3.BitVec('initialReceiver',16)
p = z3.BitVec('p',16)
q = z3.BitVec('q',16)
timestamp = z3.BitVec('timestamp', 256)

# ERC20
balances, balancesOut = ts.add_var(z3.ArraySort(z3.BitVecSort(16), z3.BitVecSort(256)), name='balances')
allowed, allowedOut = ts.add_var(z3.ArraySort(z3.BitVecSort(16), z3.ArraySort(z3.BitVecSort(16), z3.BitVecSort(256))), name='allowed')
totalSupply, totalSupplyOut = ts.add_var(z3.BitVecSort(256), name='totalSupply')

# ERC20Detailed
name, nameOut = ts.add_var(z3.StringSort(), name='name')
symbol, symbolOut = ts.add_var(z3.StringSort(), name='symbol')
decimals, decimalsOut = ts.add_var(z3.BitVecSort(8), name='decimals')

# Melon
council, councilOut = ts.add_var(z3.BitVecSort(16), name='council')
developer, developerOut = ts.add_var(z3.BitVecSort(16), name='developer')
initialSupplyMinted, initialSupplyMintedOut = ts.add_var(z3.BoolSort(), name='initialSupplyMinted')
nextMinting, nextMintingOut = ts.add_var(z3.BitVecSort(256), name='nextMinting')

# Monitoring variable, deal with once
monitoring_burn, monitoring_burnOut = ts.add_var(z3.BoolSort(), name='monitoring_burn')
monitoring_burnFrom, monitoring_burnFromOut = ts.add_var(z3.BoolSort(), name='monitoring_burnFrom')
monitoring_transferFrom, monitoring_transferFromOut = ts.add_var(z3.BoolSort(), name='monitoring_transferFrom')
monitoring_mintInflation, monitoring_mintInflationOut = ts.add_var(z3.BoolSort(), name='monitoring_mintInflation')

# function selection variable
sel_burn, sel_burnOut = ts.add_var(z3.BoolSort(), name='sel_burn')
sel_burnFrom,sel_burnFromOut = ts.add_var(z3.BoolSort(), name='sel_burnFrom')
sel_transferFrom, sel_transferFromOut = ts.add_var(z3.BoolSort(), name='sel_transferFrom')
sel_mintInflation, sel_mintInflationOut = ts.add_var(z3.BoolSort(), name='sel_mintInflation')
 
# prev
monitoring_init, monitoring_initOut = ts.add_var(z3.BoolSort(), name='monitoring_init')
# ERC20
prev_balances, prev_balancesOut = ts.add_var(z3.ArraySort(z3.BitVecSort(16), z3.BitVecSort(256)), name='prev_balances')
prev_allowed, prev_allowedOut = ts.add_var(z3.ArraySort(z3.BitVecSort(16), z3.ArraySort(z3.BitVecSort(16), z3.BitVecSort(256))), name='prev_allowed')
prev_totalSupply, prev_totalSupplyOut = ts.add_var(z3.BitVecSort(256), name='prev_totalSupply')

# ERC20Detailed
prev_name, prev_nameOut = ts.add_var(z3.StringSort(), name='prev_name')
prev_symbol, prev_symbolOut = ts.add_var(z3.StringSort(), name='prev_symbol')
prev_decimals, prev_decimalsOut = ts.add_var(z3.BitVecSort(8), name='prev_decimals')

# Melon
prev_council, prev_councilOut = ts.add_var(z3.BitVecSort(16), name='prev_council')
prev_developer, prev_developerOut = ts.add_var(z3.BitVecSort(16), name='prev_developer')
prev_initialSupplyMinted, prev_initialSupplyMintedOut = ts.add_var(z3.BoolSort(), name='prev_initialSupplyMinted')
prev_nextMinting, prev_nextMintingOut = ts.add_var(z3.BitVecSort(256), name='prev_nextMinting')

def updateAllowance(allowed, p, q, v):
    return z3.Update(allowed, p, z3.Update(allowed[p], q, v))

prev_record = z3.And(
    monitoring_initOut == False,
    prev_balancesOut == balances,
    prev_allowedOut == allowed,
    prev_totalSupplyOut == totalSupply,
    prev_nameOut == name,
    prev_symbolOut == symbol,
    prev_decimalsOut == decimals,
    prev_councilOut == council,
    prev_developerOut == developer,
    prev_initialSupplyMintedOut == initialSupplyMinted,
    prev_nextMintingOut == nextMinting,
)

In [32]:
ts.Init = z3.And(
        monitoring_init == True,
        z3.ForAll(p, balances[p]==0),
        z3.ForAll([p,q], allowed[p][q] == 0),
        totalSupply == 0,
        name == "MelonToken",
        symbol == "MLN",
        decimals == 18,
        council == 0xDEADBEEF,
        developer == sender,
        nextMinting == INFLATION_ENABLE_DATE,
        
        monitoring_burn == False,
        monitoring_burnFrom == False,
        monitoring_transferFrom == False,
        monitoring_mintInflation == False,

        sel_burn == False,
        sel_burnFrom == False,
        sel_transferFrom == False,
        sel_mintInflation == False,
    )

In [36]:
tr_burn = z3.And(
                prev_record,

                account_burn != 0,
                value_burn <= totalSupply,
                totalSupplyOut == totalSupply - value_burn,
                value_burn <= balances[account_burn],
                balancesOut == z3.Update(balances, account_burn, balances[account_burn]-value_burn),
                allowedOut == allowed,
                nameOut == name,
                symbolOut == symbol,
                decimalsOut == decimals,
                councilOut == council,
                developerOut == developer,
                initialSupplyMintedOut == initialSupplyMinted,
                nextMintingOut == nextMinting,

                monitoring_burnOut == True,
                monitoring_burnFromOut == monitoring_burnFrom,
                monitoring_transferFromOut == monitoring_transferFrom,
                monitoring_mintInflationOut == monitoring_mintInflation,

                sel_burnOut == True,
                sel_burnFromOut == False,
                sel_transferFromOut == False,
                sel_mintInflationOut == False
            )

tr_burnFrom = z3.And(
                    prev_record,
                    
                    value_burnFrom <= allowed[account_burnFrom][sender],
                    allowedOut == updateAllowance(allowed, account_burnFrom, sender, allowed[account_burnFrom][sender]-value_burnFrom),
                    account_burnFrom != 0,
                    value_burnFrom <= totalSupply,
                    totalSupplyOut == totalSupply - value_burnFrom,
                    value_burnFrom <= balances[account_burnFrom],
                    balancesOut == z3.Update(balances, account_burnFrom, balances[account_burnFrom]-value_burnFrom),

                    nameOut == name,
                    symbolOut == symbol,
                    decimalsOut == decimals,
                    councilOut == council,
                    developerOut == developer,
                    initialSupplyMintedOut == initialSupplyMinted,
                    nextMintingOut == nextMinting,

                    monitoring_burnOut == monitoring_burn,
                    monitoring_burnFromOut == True,
                    monitoring_transferFromOut == monitoring_transferFrom,
                    monitoring_mintInflationOut == monitoring_mintInflation,

                    sel_burnOut == False,
                    sel_burnFromOut == True,
                    sel_transferFromOut == False,
                    sel_mintInflationOut == False
        )

tr_transferFrom = z3.And(
                    prev_record,
                    
                    value_transferFrom <= allowed[from_transferFrom][sender],
                    allowedOut == updateAllowance(allowed, from_transferFrom, sender, allowed[from_transferFrom][sender]-value_transferFrom),
                    to_transferFrom != 0,
                    value_transferFrom <= balances[from_transferFrom],
                    balancesOut == z3.Update(balances, from_transferFrom, balances[from_transferFrom]-value_transferFrom),
                    balancesOut == z3.Update(balances, to_transferFrom, balances[to_transferFrom]+value_transferFrom),
                    balancesOut[to_transferFrom] >= balances[to_transferFrom],
                    
                    totalSupplyOut == totalSupply,
                    nameOut == name,
                    symbolOut == symbol,
                    decimalsOut == decimals,
                    councilOut == council,
                    developerOut == developer,
                    initialSupplyMintedOut == initialSupplyMinted,
                    nextMintingOut == nextMinting,

                    monitoring_burnOut == monitoring_burn,
                    monitoring_burnFromOut == monitoring_burnFrom,
                    monitoring_transferFromOut == True,
                    monitoring_mintInflationOut == monitoring_mintInflation,

                    sel_burnOut == False,
                    sel_burnFromOut == False,
                    sel_transferFromOut == True,
                    sel_mintInflationOut == False
        )
tr_mintInitialSupply = z3.And(
                    prev_record,
                    
                    initialSupplyMinted == False,
                    initialSupplyMintedOut == True,
                    allowedOut == updateAllowance(allowed, from_transferFrom, sender, allowed[from_transferFrom][sender]-value_transferFrom),
                    initialReceiver != 0,
                    totalSupplyOut == totalSupply + INITIAL_TOTAL_SUPPLY,
                    totalSupplyOut >= totalSupply,
                    balancesOut == z3.Update(balances, initialReceiver, balances[initialReceiver]+INITIAL_TOTAL_SUPPLY),
                    balancesOut[initialReceiver] >= balances[initialReceiver],
                    
                    totalSupplyOut == totalSupply,
                    nameOut == name,
                    symbolOut == symbol,
                    decimalsOut == decimals,
                    councilOut == council,
                    developerOut == developer,
                    initialSupplyMintedOut == initialSupplyMinted,
                    nextMintingOut == nextMinting,

                    monitoring_burnOut == monitoring_burn,
                    monitoring_burnFromOut == monitoring_burnFrom,
                    monitoring_transferFromOut == monitoring_transferFrom,
                    monitoring_mintInflationOut == True,

                    sel_burnOut == False,
                    sel_burnFromOut == False,
                    sel_transferFromOut == False,
                    sel_mintInflationOut == True,
        )

ts.Tr = z3.Or(
    tr_burn, 
    tr_burnFrom,
    tr_transferFrom,
    tr_mintInitialSupply
    )

In [37]:

constant_decimals = z3.Implies(z3.Not(monitoring_init), prev_decimals == decimals)
only_burn_decreases_tokens = z3.Implies(z3.Not(monitoring_init), z3.Implies(prev_totalSupply > totalSupply, z3.Or(sel_burn, sel_burnFrom)))
balance_safety = z3.Implies(z3.Not(monitoring_init), z3.Implies(prev_balances[1] > balances[1], z3.Or(sender == 1, sel_burnFrom, sel_burn, sel_transferFrom)))

Prove init => property.
proved
Prove property is inductive.
proved
Prove init => property.
proved
Prove property is inductive.
proved
Prove init => property.
proved
Prove property is inductive.
proved
